### 문제 5-1 : 카페 메뉴 도구(Tool) 호출 체인 구현
이 문제는 LangChain의 Tool Calling 기능을 학습하기 위한 기초 단계입니다. 카페 메뉴 정보를 제공하는 AI 어시스턴트를 구현하면서 다양한 데이터 소스(로컬 DB, 웹, 위키피디아)에서 정보를 검색하는 방법을 익힙니다.


In [3]:
import os
from dotenv import load_dotenv
from typing import List

# LangChain Core/Community Imports
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

# Groq과 Tavily 임포트
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch

# LangChain Agent Imports
from langchain.tools import tool
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_openai_tools_agent, AgentExecutor

# .env 환경변수 로드
load_dotenv()

# 벡터 DB 생성 함수
def build_vector_db():
    loader = TextLoader("../data/cafe_menu_data.txt", encoding="utf-8")
    docs = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    splits = text_splitter.split_documents(docs)
    embeddings = OllamaEmbeddings(model="bge-m3")
    db = FAISS.from_documents(splits, embeddings)
    db.save_local("./db/cafe_db")

# 로컬 DB 검색용 도구
@tool
def db_search_cafe_func(query: str) -> List[Document]:
    """로컬 카페 벡터 DB에서 메뉴 정보를 검색합니다. 아메리카노, 라떼 등 특정 메뉴에 대한 가격, 설명, 특징을 찾을 때 유용합니다."""
    embeddings = OllamaEmbeddings(model="bge-m3")
    db = FAISS.load_local("./db/cafe_db", embeddings, allow_dangerous_deserialization=True)
    return db.similarity_search(query)

# 위키 요약 도구
@tool
def wiki_summary(topic: str) -> str:
    """위키피디아에서 특정 주제에 대한 일반적인 정보나 요약을 검색합니다."""
    wiki = WikipediaAPIWrapper()
    return wiki.run(topic)

# 웹 검색 도구 (Tavily)
tavily_tool = TavilySearch(max_results=1)

# LLM을 ChatGroq으로 변경
llm = ChatGroq(model_name="llama3-8b-8192")

# Tools List
tools = [
    db_search_cafe_func,
    wiki_summary,
    tavily_tool
]

# ⚠️ [FIX] Prompt Template 생성 시 시스템 메시지를 한국어로 변경
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 친절한 AI 어시스턴트입니다. 모든 최종 답변은 반드시 한국어로 작성해야 합니다."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# Agent 생성
agent = create_openai_tools_agent(llm, tools, prompt)

# Agent Executor 생성
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 체인 실행 함수
def run_chain():
    question = "아메리카노의 가격과 특징은 무엇인가요?"
    response = agent_executor.invoke({"input": question})
    print("🔍 답변:", response["output"])

# 메인 실행
if __name__ == "__main__":
    # DB 생성 로직은 동일
    if not os.path.exists("./db/cafe_db/index.faiss"):
        print("✅ 벡터 DB가 없어 새로 생성합니다...")
        build_vector_db()
    else:
        print("✅ 벡터 DB가 이미 존재합니다.")

    # API 키 존재 여부 확인
    if os.getenv("GROQ_API_KEY") is None:
        print("🛑 GROQ_API_KEY 환경변수가 설정되지 않았습니다. .env 파일을 확인해주세요.")
    elif os.getenv("TAVILY_API_KEY") is None:
        print("🛑 TAVILY_API_KEY 환경변수가 설정되지 않았습니다. .env 파일을 확인해주세요.")
    else:
        print("🚀 Groq 에이전트를 실행합니다...")
        run_chain()

✅ 벡터 DB가 이미 존재합니다.
🚀 Groq 에이전트를 실행합니다...


> Entering new AgentExecutor chain...

Invoking: `db_search_cafe_func` with `{'query': '아메리카노'}`


[Document(id='246a64ee-a41b-41d0-97fe-fbf128042f2f', metadata={'source': '../data/cafe_menu_data.txt'}, page_content='1. 아메리카노\n   • 가격: ₩4,500\n   • 주요 원료: 에스프레소, 뜨거운 물\n   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.\n\n2. 카페라떼\n   • 가격: ₩5,500\n   • 주요 원료: 에스프레소, 스팀 밀크\n   • 설명: 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다. 크리미한 질감과 부드러운 맛이 특징이며, 다양한 시럽과 토핑 추가가 가능합니다. 라떼 아트로 시각적 즐거움도 제공합니다.\n\n3. 카푸치노\n   • 가격: ₩5,000\n   • 주요 원료: 에스프레소, 스팀 밀크, 우유 거품\n   • 설명: 에스프레소, 스팀 밀크, 우유 거품이 1:1:1 비율로 구성된 이탈리아 전통 커피입니다. 진한 커피 맛과 부드러운 우유 거품의 조화가 일품이며, 계피 파우더를 뿌려 제공합니다.'), Document(id='9c8f7d9b-68d1-44f3-a2c5-b68e49beccd3', metadata={'source': '../data/cafe_menu_data.txt'}, page_content='9. 아이스 아메리카노\n   • 가격: ₩4,500\n   • 주요 원료: 에스프레소, 차가운 물, 얼음\n   • 설명: 진한 에스프레소에 차가운 물과 얼음을 넣어 만든 시원한 아이스 커피입니다. 

### 문제 6-1 : 조건부 분기가 있는 메뉴 추천 시스템 ( LangGraph 사용하기)

In [12]:
import os
from dotenv import load_dotenv
from typing import List

# --- 라이브러리 임포트 ---
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch
from langchain.tools import tool
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_openai_tools_agent, AgentExecutor

# --- 환경변수 로드 ---
load_dotenv()

# --- 도구 정의 ---
@tool
def db_search_cafe_func(query: str) -> List[Document]:
    """카페 메뉴(음료, 디저트 등)의 가격, 재료, 상세 설명을 로컬 벡터 DB에서 검색합니다. '카페라떼', '티라미수' 등 메뉴 이름으로 검색할 때 사용합니다."""
    print(f"🧰 db_search_cafe_func 호출: {query}")
    embeddings = OllamaEmbeddings(model="bge-m3")
    db = FAISS.load_local("./db/cafe_db", embeddings, allow_dangerous_deserialization=True)
    return db.similarity_search(query)

@tool
def tavily_search_func(query: str) -> str:
    """'라떼의 유래', '커피의 역사'와 같은 일반적인 지식, 배경, 역사, 최신 트렌드, 뉴스 등 웹 검색이 필요한 모든 경우에 사용하는 가장 신뢰도 높은 기본 검색 도구입니다."""
    print(f"🧰 tavily_search_func 호출: {query}")
    tavily_tool = TavilySearch(max_results=3)
    results = tavily_tool.invoke(query)
    # 결과가 리스트 형태의 딕셔너리일 수 있으므로, 문자열로 변환
    return str(results)


@tool
def wiki_summary(topic: str) -> str:
    """tavily_search_func로 정보를 찾지 못했을 때만 사용하는 예비용 검색 도구입니다. 일반적인 주제에 대해 검색합니다."""
    print(f"🧰 wiki_summary 호출: {topic}")
    wiki = WikipediaAPIWrapper()
    return wiki.run(topic)


# --- ⚠️ [FIX] 환각 방지 및 한국어 답변을 강화한 최종 프롬프트 ---
def create_final_prompt():
    """환각 방지 및 한국어 답변을 강화한 최종 프롬프트 템플릿을 생성합니다."""
    system_message = """
    당신은 카페 메뉴 정보와 음식 지식을 제공하는 한국어 전문 AI 어시스턴트입니다.

    # 당신의 작업 순서:
    1. 사용자의 질문을 한국어로 분석하여 핵심 키워드를 정확히 파악합니다.
    2. 아래 '도구 사용 가이드라인'에 따라 사용할 도구를 결정합니다.
    3. 도구를 사용할 때는, **질문에서 파악한 한국어 핵심 키워드를 그대로 인자로 사용하세요.** 절대로 영어나 의미 없는 단어로 바꾸지 마세요.
    4. 질문이 여러 부분으로 나뉘어 있다면, 필요한 도구들을 순서대로 모두 호출하여 정보를 수집합니다.
    5. **매우 중요**: 만약 도구가 오류를 반환하거나 정보를 찾지 못했다면, **절대로 답을 지어내지 마세요.** 대신 "죄송하지만, 'OOO'에 대한 정보를 찾는 데 실패했습니다."라고 솔직하게 말해야 합니다.
    6. 모든 정보 수집이 완료되면, 수집된 내용을 바탕으로 논리적이고 친절한 최종 답변을 생성합니다.
    7. 답변의 마지막에는 어떤 도구를 사용했는지 "(출처: ...)" 형식으로 명시합니다.
    8. **마지막으로, 무슨 일이 있어도 모든 최종 답변은 반드시, 온전히 한국어로만 작성해야 합니다.**

    # 도구 사용 가이드라인:
    - `db_search_cafe_func`: '카페라떼', '티라미수' 등 저희 카페의 **특정 메뉴**에 대한 가격, 설명, 재료 정보가 필요할 때 사용합니다.
    - `tavily_search_func`: '라떼의 유래', '커피의 역사'와 같은 **일반적인 지식, 배경, 역사**를 검색하거나, 최신 트렌드, 뉴스 등 **웹 검색이 필요한 모든 경우**에 사용합니다. 위키피디아보다 신뢰도가 높으므로 우선적으로 사용하세요.
    - `wiki_summary`: `tavily_search_func`로 정보를 찾지 못했을 때만 예비용으로 사용하세요.
    """

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_message),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
            ("human", "{input}"),
        ]
    )
    return prompt

# --- 메인 실행 함수 ---
def main():
    """메인 실행 로직"""
    if not os.path.exists("./db/cafe_db/index.faiss"):
        print("✅ 벡터 DB가 없어 새로 생성합니다...")
        loader = TextLoader("./data/cafe_menu_data.txt", encoding="utf-8")
        docs = loader.load()
        text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        splits = text_splitter.split_documents(docs)
        embeddings = OllamaEmbeddings(model="bge-m3")
        db = FAISS.from_documents(splits, embeddings)
        db.save_local("./db/cafe_db")
    else:
        print("✅ 벡터 DB가 이미 존재합니다.")

    llm = ChatGroq(model_name="llama3-8b-8192")
    tools = [db_search_cafe_func, tavily_search_func, wiki_summary]

    prompt = create_final_prompt()
    agent = create_openai_tools_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

    print("\n--- 복합 질문 테스트 시작 ---")
    question = "카페라떼와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래에 대해서도 알려주세요."
    print(f"🤔 질문: {question}")
    
    response = agent_executor.invoke({"input": question})
    
    print("\n--- 최종 답변 ---")
    print(f"🤖 답변: {response['output']}")

if __name__ == "__main__":
    if os.getenv("GROQ_API_KEY") is None or os.getenv("TAVILY_API_KEY") is None:
        print("🛑 GROQ_API_KEY 또는 TAVILY_API_KEY 환경변수가 설정되지 않았습니다. .env 파일을 확인해주세요.")
    else:
        main()

✅ 벡터 DB가 이미 존재합니다.

--- 복합 질문 테스트 시작 ---
🤔 질문: 카페라떼와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래에 대해서도 알려주세요.


> Entering new AgentExecutor chain...

Invoking: `db_search_cafe_func` with `{'query': '카페라떼와 어울리는 디저트'}`


🧰 db_search_cafe_func 호출: 카페라떼와 어울리는 디저트
[Document(id='bf595ec5-b91e-47a1-b632-b9398e2c14b8', metadata={'source': '../data/cafe_menu_data.txt'}, page_content='4. 바닐라 라떼\n   • 가격: ₩6,000\n   • 주요 원료: 에스프레소, 스팀 밀크, 바닐라 시럽\n   • 설명: 카페라떼에 달콤한 바닐라 시럽을 더한 인기 메뉴입니다. 바닐라의 달콤함과 커피의 쌉싸름함이 조화롭게 어우러지며, 휘핑크림 토핑으로 더욱 풍성한 맛을 즐길 수 있습니다.\n\n5. 카라멜 마키아토\n   • 가격: ₩6,500\n   • 주요 원료: 에스프레소, 스팀 밀크, 카라멜 시럽, 휘핑크림\n   • 설명: 스팀 밀크 위에 에스프레소를 부어 만든 후 카라멜 시럽과 휘핑크림으로 마무리한 달콤한 커피입니다. 카라멜의 진한 단맛과 커피의 깊은 맛이 조화를 이루며, 시각적으로도 아름다운 층을 형성합니다.'), Document(id='246a64ee-a41b-41d0-97fe-fbf128042f2f', metadata={'source': '../data/cafe_menu_data.txt'}, page_content='1. 아메리카노\n   • 가격: ₩4,500\n   • 주요 원료: 에스프레소, 뜨거운 물\n   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.\